# Gensim 패키지
- Python으로 작성된 오픈 소스 라이브러리로, 자연어 처리와 관련된 다양한 기능을 제공한다.
- 주요 기능
    - **Word Embeddings**
        - word2vec, fastext, doc2vec 등 다양한 word embedding 모델을 제공
    - **토픽 모델링 (Topic Modeling)**
        - LDA등 문장의 주제를 파악하는 모델 제공
    - **텍스트/word 유사도 계산**
    - **문서 군집화**
        - 비슷한 주제의 문서들을 군집화.
    - 다양한 dataset과 pretrained model 제공
        - https://github.com/piskvorky/gensim-data
- https://radimrehurek.com/gensim/

## 설치
- `pip install gensim`

In [1]:
%pip install gensim

  Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl (24.0 MB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl (45.9 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl (38 kB)

  Attempting uninstall: numpy

    Found existing installation: numpy 2.2.5

    Uninstalling numpy-2.2.5:

   -------- ------------------------------- 1/5 [numpy]
   -------- ------------------------------- 1/5 [numpy]
   -------- ------------------------------- 1/5 [numpy]
   -------- ------------------------------- 1/5 [numpy]
   -------- --

# Word2Vec 학습

- gensim.models.Word2Vec
- 주요 파라미터
    - sentences
        -  학습에 사용할 문서의 리스트. 각 문서의 단어들을 리스트로 묶고 그 문서들을 리스트로 묶은 중첩 리스트.
        - 예시: \[\['word1', 'word2', 'word3'], \['word4', 'word5']]
    - vector_size
        -  embedding vector 크기. 기본값: 100
    - window
        -  context window 크기. 중심단어를 기준으로 좌우 몇개의 단어를 확인하는지 크기. 기본값: 5
    - min_count
        - 이 설정보다 낮은 빈도로 등장하는 단어는 무시한다. 데이터 노이즈를 줄이는데 도움이된다. 기본값: 5
    - sg
        - 모델 아키텍처 결정.
        - `0`: CBOW, `1`: Skip-gram. 기본값: 0
    - epochs
        - epochs 수 설정. 기본값: 5
    - alpha
        - initial leaning rate. 기본값: 0.025
    - min_alpha
        - 최소 learning rate. 기본값: 0.0001
        - epoch 마다 learning rate를 alpha 에서 min_alpha 까지 선형적으로 줄여나간다.
    - workers
        -  사용 Thread 수. 기본값: 3

## 학습(Train)
1. Word2Vec 의 initializer에 sentences를 넣어 한번에 학습한다.
2. Word2Vec 클래스에 학습 설정을 하고 `train()` 메소드를 이용해 학습한다.
    - epoch 단위로 작업을 할 경우 사용

In [9]:
# 샘플 텍스트 데이터
sentences = [
    "Natural language processing is an exciting field of study",
    "Word embeddings are a type of word representation",
    "Gensim is a powerful library for text processing",
    "Word2Vec creates vector representations of words", 
    "Gensim runs on Linux, Windows and OS X, as well as any other platform that supports Python and NumPy."
    "All Gensim source code is hosted on Github under the GNU LGPL license, maintained by its open source community.",
    "For commercial arrangements, see Business Support.",
    "Gensim can process arbitrarily large corpora, using data-streamed algorithms.",
    "There are no \"dataset must fit in RAM\" limitations."
]

In [12]:
import re
import nltk
def tokenizer(docs):
    # 소문자로 모두 변환.
    # 알파벳, 숫자, _를 제외한 모든 문자들을 제거
    # 단어(어절)단위 토크화화
    return [nltk.word_tokenize(re.sub(r"[^\w\s]", "", doc.lower())) for doc in docs]
            
# [^\w]  - \w\s(공백문자들) 를 제외한 나머지(^)들을 찾아라.

In [14]:
import re
tokens = tokenizer(sentences)

In [15]:
import os 
os.cpu_count()

16

In [ ]:
# train
## Word2Vec 객체를 생성: 학습데이터, epoch -> 객체 생성할 때 모델을 학습. 학습결과 모델을 반환.
from gensim.models import Word2Vec
import os
model1 = Word2Vec(
    sentences=tokens, # 학습 시킬 데이터.
    vector_size=10,   # embedding vector의 차원(한개 단어에서 몇개 feature를 추출할지.)
    window=2,         # window size 설정. 주변단어의 개수.
    min_count=1,      # 최소 출연 빈도수. 
    epochs=10,
    workers=os.cpu_count()         # 병렬처리 개수. 
)

In [ ]:
# 학습데이터, epoch을 설정하지 않음. => 학습안된 모델을 반환.
model2 = Word2Vec(vector_size=10, window=2, min_count=1, workers=os.cpu_count())
# model에 vocab을 설정.
model2.build_vocab(tokens)
# 학습
epoch=10
for e in range(epoch):
    model2.train(
        tokens,  # 학습데이터
        total_examples=model2.corpus_count, # 학습데이터(문서) 개수
        epochs=1,
        compute_loss=True    # 학습이 끝나면 loss를 계산.
    )
    loss = model2.get_latest_training_loss() # train() 시 계산된 loss를 조회.
    print(f"{e} epoch loss: {loss}")

0 epoch loss: 1449.962646484375
1 epoch loss: 1368.042236328125
2 epoch loss: 1291.6473388671875
3 epoch loss: 1328.950439453125
4 epoch loss: 1373.55517578125
5 epoch loss: 1295.62939453125
6 epoch loss: 1308.69287109375
7 epoch loss: 1341.904541015625
8 epoch loss: 1299.2216796875
9 epoch loss: 1398.2099609375


## 학습 후 결과 조회

- **KeyedVectors 조회**
    - KeyedVectors는 단어와 vector를 매핑한 객체로 embedding vector를 이용한 다양한 조회를 지원한다.
    - model.wv 로 조회해서 사용.
- **Embedding Vector 조회**
  - model.wv.vectors
- **단어 목록 조회**
    - model.wv.index_to_key, model.wv.key_to_index
- **단어 벡터 조회**
    - model.wv[word]: 특정 단어의 vector반환
- **Vocab에 대상 단어가 있는지 확인**
    - "대상단어" in model.wv
- **유사단어들 찾기**
    - model.wv.most_similar(word)
- **단어간 유사도 비교**
    - model.wv.similarity(word1, word2)
- 유사도를 계산할 때 **코사인 유사도(Cosine Similarity)** 를 사용한다.

> # 코사인 유사도
> - 두 벡터 간의 유사성을 측정하는 중요한 방법 중 하나.
> - 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 계산한다. 이때 벡터의 **크기는 결과에 영향을 미치지 않고, 오직 방향만이 중요**하다.
> ## 공식
> 
> $$ similarity = cos(\theta) = \frac{A⋅B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n}{A_i×B_i}}{\sqrt{\sum_{i=1}^{n}(A_i)^2}×\sqrt{\sum_{i=1}^{n}(B_i)^2}} $$
> 
> ## 결과 해석
> 
> - **값의 범위**: -1에서 1 사이의 값을 가집니다
>   - 1: 두 벡터가 완전히 동일한 방향 (0도의 cosine 값)
>   - 0: 두 벡터가 직교 (90도의 cosine 값)
>   - -1: 두 벡터가 정반대 방향 (180도의 cosine 값)
> 
> ![cosine_similarity](figures/gensim_consin_sim.png)
>
> ## Python 코사인 유사도 계산
> ```python
> from numpy import dot
> from numpy.linalg import norm
> 
> def cosine_similarity(A, B):
>     return dot(A, B)/(norm(A)*norm(B))
> ```



In [24]:
model1.wv  # KeyedVectors -> 토큰-embedding vector

In [25]:
model1.wv.index_to_key # vocab: token_id -> token(단어)

['gensim',
 'is',
 'of',
 'for',
 'processing',
 'source',
 'as',
 'on',
 'word',
 'are',
 'a',
 'and',
 'words',
 'runs',
 'linux',
 'os',
 'windows',
 'vector',
 'x',
 'well',
 'any',
 'representations',
 'limitations',
 'creates',
 'word2vec',
 'text',
 'platform',
 'library',
 'powerful',
 'representation',
 'type',
 'embeddings',
 'study',
 'field',
 'exciting',
 'an',
 'language',
 'other',
 'that',
 'ram',
 'see',
 'support',
 'can',
 'process',
 'arbitrarily',
 'large',
 'corpora',
 'using',
 'datastreamed',
 'algorithms',
 'there',
 'no',
 'dataset',
 'must',
 'fit',
 'in',
 'business',
 'arrangements',
 'supports',
 'commercial',
 'python',
 'numpyall',
 'code',
 'hosted',
 'github',
 'under',
 'the',
 'gnu',
 'lgpl',
 'license',
 'maintained',
 'by',
 'its',
 'open',
 'community',
 'natural']

In [26]:
model1.wv.key_to_index

{'gensim': 0,
 'is': 1,
 'of': 2,
 'for': 3,
 'processing': 4,
 'source': 5,
 'as': 6,
 'on': 7,
 'word': 8,
 'are': 9,
 'a': 10,
 'and': 11,
 'words': 12,
 'runs': 13,
 'linux': 14,
 'os': 15,
 'windows': 16,
 'vector': 17,
 'x': 18,
 'well': 19,
 'any': 20,
 'representations': 21,
 'limitations': 22,
 'creates': 23,
 'word2vec': 24,
 'text': 25,
 'platform': 26,
 'library': 27,
 'powerful': 28,
 'representation': 29,
 'type': 30,
 'embeddings': 31,
 'study': 32,
 'field': 33,
 'exciting': 34,
 'an': 35,
 'language': 36,
 'other': 37,
 'that': 38,
 'ram': 39,
 'see': 40,
 'support': 41,
 'can': 42,
 'process': 43,
 'arbitrarily': 44,
 'large': 45,
 'corpora': 46,
 'using': 47,
 'datastreamed': 48,
 'algorithms': 49,
 'there': 50,
 'no': 51,
 'dataset': 52,
 'must': 53,
 'fit': 54,
 'in': 55,
 'business': 56,
 'arrangements': 57,
 'supports': 58,
 'commercial': 59,
 'python': 60,
 'numpyall': 61,
 'code': 62,
 'hosted': 63,
 'github': 64,
 'under': 65,
 'the': 66,
 'gnu': 67,
 'lgpl': 

In [28]:
# 특정 단어의 embedding vector 를 조회
model1.wv['gensim']

array([-0.00530078,  0.00215998,  0.0511257 ,  0.08994552, -0.0929675 ,
       -0.07124555,  0.06510045,  0.08978292, -0.05092729, -0.0380263 ],
      dtype=float32)

In [ ]:
# model1.wv['안녕'] - vocab에 특정 단어가 있는지 여부
'안녕' in model1.wv , 'gensim' in model1.wv

(False, True)

In [34]:
word = '안녕'
word = "numpyall"
if word in model1.wv:
    print(model1.wv[word])
else:
    print("없는 단어")

[ 0.05893448  0.01508251 -0.00718124  0.09339169 -0.04917641 -0.00835453
  0.09189787  0.06754155  0.01489585 -0.08890103]


In [ ]:
# 유사도 검사.
model1.wv.most_similar("gensim")  # gensim과 가장 유사한 단어를 순서대로 10개를 반환.
#(단어, 유사도) 유사도 -1 ~ 1 

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195193886756897),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

In [36]:
model1.wv.most_similar("gensim", topn=3)

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729)]

In [37]:
# 단어간의 유사도
model1.wv.similarity("gensim", "study")

0.7138967

## 모델 저장 및 로딩

### 모델 저장, 로딩
- `model.save('저장파일 경로')`
  - gensim 자체 포맷으로 저장된다.
- `gensim.models.Word2Vec.load('저장파일 경로')`
  - `model.save()`로 저장된 모델을 Loading한다.

### Word Embedding Vector만 저장 및 로드
- `KeyedVectors` 를 이용해 저장한다.
    - `model.wv.save_word2vec_format('저장경로', binary=True|False)`
        - binary=True: binary 파일로 저장한다. 용량이 작은 대신 내용확인이 안된다.
        - binary=False: csv(공백구분자) 형식 text로 저장한다. 내용을 확인할 수있지만 파일용량이 크다.
- `KeyedVectors.load_word2vec_format("저장경로", binary=True|False)`
    - 저장시 binary에 맞춰 읽는다.

## Pretrained 모델 사용하기
- https://github.com/Kyubyong/wordvectors

# 빅카인즈 뉴스 데이터를 이용한 Word2Vec 학습
- 빅카인즈
    - 한국언론진흥재단에서 운영하는 뉴스빅데이터 분석 서비스 사이트
- 빅카인즈에서 특정 분야의 기사들을 수집해서 학습시킨다.
    - https://www.bigkinds.or.kr/
    - 회원가입 (구글, 네이버, 카카오 계정으로 가입 가능) 후 로그인
    - 뉴스분석 > 뉴스검색$\cdot$분석 클릭
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds1.png)
    - 기간, 언론사, 분류, 상세검색 등 검색 조건입력 후 조회
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds2.png)
    - 결과 다운로드
        - step3 분석결과및 시각화 -> 맨 아래 `엑셀다운로드` 클릭 
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds3.png)